# Self Driving Car Engineer Nanodegree

## Project: Build a Vehicle Detection Pipeline

In this project, the goal is to write a software pipeline to detect vehicles in a video (start with the test_video.mp4 and later implement on full project_video.mp4).

The goals / steps of this project are the following:

* Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier
* Optionally, you can also apply a color transform and append binned color features, as well as histograms of color, to your HOG feature vector. 
* Note: for those first two steps don't forget to normalize your features and randomize a selection for training and testing.
* Implement a sliding-window technique and use your trained classifier to search for vehicles in images.
* Run your pipeline on a video stream (start with the test_video.mp4 and later implement on full project_video.mp4) and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.
* Estimate a bounding box for vehicles detected.

Here are links to the labeled data for [vehicle](https://s3.amazonaws.com/udacity-sdc/Vehicle_Tracking/vehicles.zip) and [non-vehicle](https://s3.amazonaws.com/udacity-sdc/Vehicle_Tracking/non-vehicles.zip) examples to train your classifier.  These example images come from a combination of the [GTI vehicle image database](http://www.gti.ssr.upm.es/data/Vehicle_database.html), the [KITTI vision benchmark suite](http://www.cvlibs.net/datasets/kitti/), and examples extracted from the project video itself.   You are welcome and encouraged to take advantage of the recently released [Udacity labeled dataset](https://github.com/udacity/self-driving-car/tree/master/annotations) to augment your training data.

---

## Step 0: Preparation

In [2]:
import glob
import time
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.mplot3d import Axes3D
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from scipy.ndimage.measurements import label
from moviepy.editor import VideoFileClip
from IPython.display import HTML

plt.style.use('fivethirtyeight')
%matplotlib inline

In [ ]:
# Load data
# TODO: Read images with cv2 and be aware of BGR format

In [ ]:
###################################################################
# HELPER FUNCTIONS
###################################################################

# BOUNDING BOXES
# Draw bounding boxes on an image
def draw_boxes(img, bboxes, color=(255, 0, 0), thick=6):
"""
img: image to draw on
bboxes: list of tuples with coordinates for top left and bottom right corner
color: color of the bounding boxes as RGB tuple
thick: thickness of the line of the bounding boxes
return: copy of the image with drawn bounding boxes
"""
    imcopy = np.copy(img)
    
    for box in bboxes:
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    return imcopy

# PLOT COLOR SPACE IN 3D
# Compute 3D scatter plot to visualize image pixels in color space
def plot3d(img, colors_rgb, axis_labels=list("RGB"), axis_limits=((0,255), (0,255), (0,255))):
"""
img: the image which is to analyze 
colors_rgb: colors for the pixel values
axis_labels: list of label names for the axes
axis_limits: tuple with lower and upper limits of the axes
return: Axes3D object
"""
    # Create figure and 3D axes
    fig = plt.figure(figsize=(8,8))
    ax = Axes3D(fig)
    
    # Set axis limits
    ax.set_xlim(*axis_limits[0])
    ax.set_ylim(*axis_limits[1])
    ax.set_zlim(*axis_limits[2])
    
    # Set axis labels and sizes
    ax.tick_params(axis="both", which="major", labelsize=14, pad=8)
    ax.set_xlabel(axis_labels[0], fontsize=16, labelpad=16)
    ax.set_ylabel(axis_labels[1], fontsize=16, labelpad=16)
    ax.set_zlabel(axis_labels[2], fontsize=16, labelpad=16)
    
    # Plot pixel values with colors given in colors_rgb
    ax.scatter(
        img[:,:,0].ravel(),
        img[:,:,1].ravel(),
        img[:,:,2].ravel(),
        c=colors_rgb.reshape((-1,3)), edgecolors='none')
    
    return ax

In [ ]:
###################################################################
# FEATURE EXTRACTION
###################################################################

# Dictionary for color transform mapping on default RGB images
RGB_COLOR_TRANSFORMS = {"HSV": cv2.COLOR_RGB2HSV, "HLS": cv2.COLOR_RGB2HLS,
                       "LUV": cv2.RGB2LUV, "YUV": cv2.RGB2YUV, "YCrCb": cv2.COLOR_RGB2YCrCb}
BGR_COLOR_TRANSFORMS = {"RGB": cv2.COLOR_BGR2RGB, "HSV": cv2.COLOR_BGR2HSV, "HLS": cv2.COLOR_BGR2HLS,
                       "LUV": cv2.BGR2LUV, "YUV": cv2.BGR2YUV, "YCrCb": cv2.COLOR_BGR2YCrCb}
# TODO: Maybe add more colorspaces

# COLOR HISTOGRAM
# Compute color histogram features
def color_hist(img, nbins=32, bins_range=(0, 256)):
"""
img: image to compute the color histogram on
nbins: number of bins for the histogram
bins_range: a tuple with the lower and upper range of the bins
return: histogram for each channel of the image, array with centers of the 
        bins and feature vector of the histograms
"""
    channel_histograms = []
    # Compute the histogram of each channel separately
    for channel in range(img.shape[-1]):
        hist = np.histogram(img[:,:,channel], bins=nbins, range=bins_range)
        channel_histograms.append(hist)
    
    # Generating bin centers
    bin_edges = channel_histograms[0][1]
    bin_centers = (bin_edges[1:] + bin_edges[0:len(bin_edges)-1])/2
    
    # Concatenate the histograms into a single feature vector
    # TODO: concatenate histogram features
    # TODO: retunr channel histograms, bin centers and feature vector
    return channel_histograms

# SPATIAL BINNING
# Compute spatial color features
def bin_spatial(img, color_space=None, size=(32,32), color_transforms=RGB_COLOR_TRANSFORMS):
"""
img: image that should be converted into spatial features
color_space: color space to take the features of
size: size of spatial feature area
return: spatial feature vector
"""
    # Convert image to new color space
    if color_space != None:
        feature_img = cv2.cvtColor(img, color_transforms[color_space])
    else:
        feature_img = np.copy(img)
    # Create spatial color feature vector
    features = cv2.resize(feature_img, size).ravel()
    
    return features
    
# HOG FEATURES
# Compute histogram of oriented gradients (hog) features
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
"""
img: image where features should be extracted
orient: number of different gradient orientations to consider
pix_per_cell: size of the cells
cell_per_block: number of cells for each block
vis: should the hog feature visualization be returned
feature_vec: features should be returned as vector
return: features or features and an image with a hog feature visualization
"""
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                 cells_per_block=(cell_per_block, cell_per_block), visualize=True, 
                                 feature_vector=feature_vec)
        return features, hog_image
    else:
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                      cells_per_block=(cell_per_block, cell_per_block), visualize=False, feature_vector=feature_vec)
        return features

# COMBINE
# Extract features from a list of images 
def extract_features(imgs, cspace=None, spatial_size=(32, 32), hist_bins=32, hist_range=(0,256), 
                     color_transforms=RGB_COLOR_TRANSFORMS):
"""
imgs: list of images to extract features on
cspace: colorspace on which to operate
spatial_size: size for the spatial binning
hist_bins: number of bins for color histograms
hist_range: range of the histograms
return: list of feature vectors
"""
    # Create a list to append feature vectors to
    features = []
    
    # Transform colorspace if necessary
    for img in imgs:
        if cspace != None:
            feature_img = cv2.cvtColor(img, color_transforms[cspace])
        else:
            feature_img = np.copy(img)
        # Apply spatial binning
        bin_features = bin_spatial(feature_img, size=spatial_size)
        # Apply color histogram
        hist_features = color_hist(feature_img, nbins=hist_bins, bins_range=hist_range)
        # Append the feature vector to the list
        features.append(np.concatenate((bin_features, hist_features), axis=0))
    
    return features

# NORMALIZE
# 
def normalize_features(features):
"""
features: list of feature vectors
return: normalized list of feature vectors
"""
    # Fit a per column scaler
    X_scaler = StandardScaler().fit(features)
    # Apply the scaler to the features
    scaled_X = X_scaler.transform(features)
    
    return scaled_X

In [ ]:
# Example Code for some of the Functions
# Read a color image
img = cv2.imread("000275.png")

# Select a small fraction of pixels to plot by subsampling it
scale = max(img.shape[0], img.shape[1], 64) / 64  # at most 64 rows and columns
img_small = cv2.resize(img, (np.int(img.shape[1] / scale), np.int(img.shape[0] / scale)),
                       interpolation=cv2.INTER_NEAREST)

# Convert subsampled image to desired color space(s)
img_small_RGB = cv2.cvtColor(img_small, cv2.COLOR_BGR2RGB)  # OpenCV uses BGR, matplotlib likes RGB
img_small_HSV = cv2.cvtColor(img_small, cv2.COLOR_BGR2HSV)
img_small_rgb = img_small_RGB / 255.  # scaled to [0, 1], only for plotting
# Plot and show
plot3d(img_small_RGB, img_small_rgb)
plt.show()

plot3d(img_small_HSV, img_small_rgb, axis_labels=list("HSV"))
plt.show()

# DATA EXPLORATION
images = glob.glob('*.jpeg')
cars = []
notcars = []

for image in images:
    if 'image' in image or 'extra' in image:
        notcars.append(image)
    else:
        cars.append(image)
        
# Define a function to return some characteristics of the dataset 
def data_look(car_list, notcar_list):
    data_dict = {}
    # Define a key in data_dict "n_cars" and store the number of car images
    data_dict["n_cars"] = len(car_list)
    # Define a key "n_notcars" and store the number of notcar images
    data_dict["n_notcars"] = len(notcar_list)
    # Read in a test image, either car or notcar
    img = mpimg.imread(car_list[0])
    # Define a key "image_shape" and store the test image shape 3-tuple
    data_dict["image_shape"] = img.shape
    # Define a key "data_type" and store the data type of the test image.
    data_dict["data_type"] = img.dtype
    # Return data_dict
    return data_dict
    
data_info = data_look(cars, notcars)

print('Your function returned a count of', 
      data_info["n_cars"], ' cars and', 
      data_info["n_notcars"], ' non-cars')
print('of size: ',data_info["image_shape"], ' and data type:', 
      data_info["data_type"])
# Just for fun choose random car / not-car indices and plot example images   
car_ind = np.random.randint(0, len(cars))
notcar_ind = np.random.randint(0, len(notcars))
    
# Read in car / not-car images
car_image = mpimg.imread(cars[car_ind])
notcar_image = mpimg.imread(notcars[notcar_ind])


# Plot the examples
fig = plt.figure()
plt.subplot(121)
plt.imshow(car_image)
plt.title('Example Car Image')
plt.subplot(122)
plt.imshow(notcar_image)
plt.title('Example Not-car Image')

# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)
car_ind = np.random.randint(0, len(cars))

# Add heat to each box in box list
heat = add_heat(heat,box_list)
    
# Apply threshold to help remove false positives
heat = apply_threshold(heat,1)

# Visualize the heatmap when displaying    
heatmap = np.clip(heat, 0, 255)

# Find final boxes from heatmap using label function
labels = label(heatmap)
draw_img = draw_labeled_bboxes(np.copy(image), labels)


### TODO: Tweak these parameters and see how the results change.
color_space = 'HLS' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 12  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16) # Spatial binning dimensions
hist_bins = 32    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = False # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [400, None] # Min and max in y to search in slide_window()


In [ ]:
###################################################################
# CLASSIFIER
###################################################################

# LINEAR SVM
# Build a linear SVM classifier
def build_linear_svm(train, parameters):
"""
train: tuple of data for training of the classifier
parameters: dictionary of parameters for grid search
return: classifier and best parameters
"""
    svr = svm.SVC()
    # Creates the grid of possible parameter combinations
    clf = grid_search.GridSearchCV(svr, parameters)
    # Tries all parameter combinations and returns the fitted classifier
    clf.fit(*train)
    return clf, clf.best_params_

# TODO: Implement other classifiers

In [ ]:
# Color Classify Parameters:
Spatial = 50
Histbin = 32
# --> 97.5% Test Accuracy

# HOG Classify Parameters:
Colorspace = 'HLS'
Orient = 12
Pix_per_cell = 8
Cell_per_block = 2
Hog_channel = "ALL"
# --> 99% Test Accuracy

In [ ]:
###################################################################
# OBJECT DETECTION
###################################################################

# SLIDING WINDOW
# Slide a window over the image that takes those sub samples for classification
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                 xy_window=(64,64), xy_overlap=(0.5,0.5)):
"""
img: image that should be scanned for objects
x_start_stop: start and stop positions for the scanning area in x direction
y_start_stop: start and stop positions for the scanning area in y direction
xy_window: dimensions of the sliding window
xy_overlap: overlap of windows in x and y direction
return: list of tuples with window start and end positions
"""
    # Set start stop positions if not defined
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    it y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
        
    # Compute the span of the region to be searched
    x_span = np.int(x_start_stop[1] - x_start_stop[0])
    y_span = np.int(y_start_stop[1] - y_start_stop[0])
    
    # Compute the number of pixels per step in x|y
    step_pix_x = xy_window[0] * (1 - xy_overlap[0])
    step_pix_y = xy_window[1] * (1 - xy_overlap[1])
    
    # Compute the number of windows in x|y
    x_buffer = np.int(xy_window[0] * xy_overlap[0])
    y_buffer = np.int(xy_window[1] * xy_overlap[1])
    n_windows_x = np.int((x_span - x_buffer) / step_pix_x)
    n_windows_y = np.int((y_span - y_buffer) / step_pix_y)
    
    # Initialize a list to append window positions
    window_list = []
    
    # Loop through finding x and y window positions
    for step_y in range(n_windows_y):
        for step_x in range(n_windows_x):
            # Calculate each window position
            start_x = np.int(step_x * step_pix_x + x_start_stop[0])
            start_y = np.int(step_y * step_pix_y + y_start_stop[0])
            end_x = np.int(start_x + xy_window[0])
            end_y = np.int(start_y + xy_window[1])
            # Append position to list
            window_list.append(((start_x, start_y), (end_x, end_y)))
    return window_list

# HOG SUBSAMPLING WINDOW SEARCH
# Extract features using hog sub-sampling and make predictions
def find_cars(img, y_start_stop, x_start_stop, scale, svc, X_scaler, window=64, orient, pix_per_cell, cell_per_block, 
              cells_per_step=2, spatial_size, hist_bins, color=(255, 0, 0), thick=6, cspace=None, color_transforms=RGB_COLOR_TRANSFORMS):
"""
img:
y_start_stop:
x_start_stop:
scale:
svc:
X_scaler:
window:
orient:
pix_per_cell:
cell_per_block:
cells_per_step: instead of overlap define how many cells to step
spatial_size:
hist_bins:
color: color of the bounding box lines
thick: thickness of the bounding box lines
cspace:
color_transforms: 
return:
"""
    draw_img = np.copy(img)
    # TODO: Check whether following line is necessary and if better to put in if-statement,
    # e.g. if image already is 0 to 1 scale
    img = img.astype(np.float32)/255
    
    # TODO: check whether values of start stop are None and adapt accordingly
    img_search = img[y_start_stop[0]:y_start_stop[1], x_start_stop[0]:x_start_stop[1], :]
    if cspace != None:
        feature_img = cv2.cvtColor(img_search, color_transforms[cspace])
    else:
        feature_img = img_search
        
    if scale != 1:
        img_shape = feature_img.shape
        feature_img = cv2.resize(feature_img, (np.int(img_shape[1]/scale), np.int(img_shape[0]/scale)))
    
    # TODO: Check if split to channels is necessary
    ch1 = feature_img[:,:,0]
    ch2 = feature_img[:,:,1]
    ch3 = feature_img[:,:,2]
    
    # Define blocks and steps
    n_blocks_x = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1 # TODO: Understand why it is like this and what celss, blocks and windows are
    n_blocks_y = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1
    n_feat_per_block = orient * np.square(cell_per_block) # TODO: Check if np.square is the right function
    n_blocks_per_window = (window // pix_per_cell) - cell_per_block + 1
    n_steps_x = (n_blocks_x - n_blocks_per_window) // cells_per_step
    n_steps_y = (n_blocks_y - n_blocks_per_window) // cells_per_step
    
    # Compute individual channel hog features
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    for step_x in range(n_steps_x):
        for step_y in range(n_steps_y):
            pos_y = step_y * cells_per_step
            pos_x = step_x * cells_per_step
            
            # Extract hog for this patch
            # TODO: Understand why it works like this and what n_blocks_per_windo does here
            hog_feat1 = hog1[pos_y:pos_y + n_blocks_per_window, pos_x:pos_x + n_blocks_per_window].ravel() 
            hog_feat2 = hog2[pos_y:pos_y + n_blocks_per_window, pos_x:pos_x + n_blocks_per_window].ravel()
            hog_feat3 = hog3[pos_y:pos_y + n_blocks_per_window, pos_x:pos_x + n_blocks_per_window].ravel()
            
            hog_features = np.hstack((hog1, hog2, hog3)) # TODO: Check if vstack works and if not why not
            
            x_left = pos_x * pix_per_cell
            y_top = pos_y * pix_per_cell
            
            # Extract the image patch
            sub_img = cv2.resize(feature_img[y_top:y_top+window, x_left:x_left+window])
            
            # Get color features
            spatial_features = bin_spatial(sub_img, size=spatial_size)
            hist_features = color_hist(sub_img, nbins=hist_bins)
            
            # Scale features and make a prediction
            # TODO: Check if vstack works
            features = X_scaler.transform(np.hstacl((spatial_features, hist_features, hog_features)).reshape(1, -1))
            prediction = svc.predict(features)
            
            # If prediction was car (1)
            if prediction == 1:
                box_left = np.int(x_left * scale)
                box_top = np.int(y_top * scale)
                win_draw = np.int(window * scale)
                cv2.rectangle(draw_img, (box_left, y_top+y_start_stop[0]), 
                              (box_left+win_draw, y_top+win_draw+y_start_stop[0]), color, thick)
            
            return draw_img

# SEARCH OVER ALL WINDOWS
# Extract features from a single image window
def single_img_features(img, color_space=None, spatial_size=(32,32), hist_bins=32, orient=9, pix_per_cell=8,
                        cell_per_block=2, hog_channel=0, spatial_feat=True, hist_feat=True, hog_feat=True, 
                        color_transforms=RGB_COLOR_TRANSFORMS):
"""
img: 
color_space: 
spatial_size: 
hist_bins: 
orient: 
pix_per_cell: 
cell_per_block: 
hog_channel: 
spatial_feat: 
hist_feat: 
hog_feat: 
color_transforms: 
return: 
"""
    # Empty list to receive features
    features = []
    # Apply color conversion
    if color_space != None:
        feature_img = cv2.cvtColor(img, color_transforms[color_space])
    else:
        feature_img = np.copy(img)
    # Compute spatial features
    if spatial_feat:
        spatial_features = bin_spatial(feature_img, size=spatial_size)
        features.append(spatial_features)
    # Compute histogram features
    if hist_feat:
        hist_features = color_hist(feature_img, nbins=hist_bins)
        features.append(hist_features)
    # Compute hog features
    if hog_feat:
        if hog_channel == "ALL":
            hog_features = []
            for channel in range(feature_img.shape[2]):
                hog_features.extend(get_hog_features(feature_img[:,:,channel], orient, pix_per_cell, cell_per_block,
                                                    vis=False, feature_vec=True))
        else:
            hog_features = get_hog_features(feature_img[:,:,hog_channel], orient, pix_per_cell, cell_per_block, 
                                            vis=False, feature_vec=True)
        features.appen(hog_features)
    
    return np.concatenate(features)
        
# OBJECT SEARCH
# Search in windows for target objects
def search_windows(img, windows, clf, scaler, color_space=None, window_size=(64,64), spatial_size=(32,32), hist_bins=32, 
                   hist_range(0, 256), orient=9, pix_per_cell=8, cell_per_block=2, hog_channel=0, spatial_feat=True, 
                   hist_feat=True, hog_feat=True, color_transforms=RGB_COLOR_TRANSFORMS):
"""
img:
windows:
clf:
scaler:
color_space:
window_size:
spatial_size:
hist_bins:
hist_range:
orient:
pix_per_cell:
cell_per_block:
hog_channel:
spatial_feat:
hist_feat:
hog_feat:
color_transforms:
return:
"""
    # Create an empty list to receive positive detection windows
    on_windows = []
    # Iterate over all windows in the list
    for window in windows:
        # Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0], window[1][0]], window_size) # TODO: Check whether window_size is necessary, what it does and why not just set it to (64,64)
        # Extract features for that window
        features = single_img_features(test_img, color_space=color_space, spatial_size=spatial_size, hist_bins=hist_bins,
                                      orient=orient, pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                                      hog_channel=hog_channel, spatial_feat=spatial_feat, hist_feat=hist_feat,
                                      hog_feat=hog_feat, color_transforms=color_transforms)
        # Scale extracted features
        features = scaler.transform(np.array(features).reshape(1, -1)) # TODO: Check if vstack works
        # Predict the class
        prediction = clf.predict(features)
        # If prediction is car (1) save window
        if prediction == 1:
            on_windows.append(window)
        
        return on_windows
    
# HEATMAP
# Add heat values to heatmap for pixels within bounding box
def add_heat(heatmap, bbox_list):
"""
heatmap: image with pixel values representing heat
bbox_list: list of bounding box coordinates as tuples
return: heatmap with updated heat values
"""
    # Iterate through list of bounding boxes
    for box in bbox_list:
        # Add 1 for all pixels inside each box
        # Assuming each box takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
        
    return heatmap

# Zero values below threshold
def apply_threshold(heatmap, threshold):
"""
heatmap: image with pixel values representing heat
threshold: threshold value for zeroing values
return: thresholded heatmap
"""
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    
    return heatmap

# Draw bounding boxes around identified objects
def draw_labeled_bboxes(img, labels, color=(255, 0, 0), thick=6):
"""
img: image to draw on
labels: 
color: color of the bounding boxes
thick: line thickness of the bounding boxes
return: image with bounding boxes
"""
    # Iterate through all detected cars
    for car_number in range(1, len(labels) + 1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzero_y = np.arary(nonzero[0])
        nonzero_x = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzero_x), np.min(nonzero_y)), (np.max(nonzero_x), np.max(nonzero_y)))
        # Draw box on the image
        cv2.rectangle(img, bbox[0], bbox[1], color, thick)
    
    return img

**As an optional challenge** Once you have a working pipeline for vehicle detection, add in your lane-finding algorithm from the last project to do simultaneous lane-finding and vehicle detection!

**If you're feeling ambitious** (also totally optional though), don't stop there!  We encourage you to go out and take video of your own, and show us how you would implement this project on a new video!

### Plan for implementation:
* Decide on feature combination (color and gradient)
* Train classifier
* Sliding window for detection or maybe hog subsampling !?

#### Proposal for Prediction Pipeline
The pipeline might vary for CNN classifier, but for decision trees, naive bayes, support vector machines, and neural networks (on feature vectors not image information) the pipeline should follow these steps:
* Read in image
* Extract features
* Normalize feature vector
* Feed into classifier

#### Proposal for Training Pipeline
* Read in images
* Extract features
* Normalize features
* Train classifier


#### To be explored
* Make use of the sklearn pipeline framework similar to the traffic sign project example
* Have different parameter configurations explored
* Have a look at SURF and ? (computer vision in Sweden) features
* Streaming video in notebook

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [6]:
%%HTML
<video width="960" height="540" controls>
  <source src="test_video.mp4">
</video>